In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorAssembler, RFormula, Bucketizer, StandardScaler, OneHotEncoder
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.stat import Correlation
from google.colab import drive


spark = SparkSession.builder \
    .appName("EnhancedRentalPriceClassification") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
    .getOrCreate()



print(f"Spark Version: {spark.version}")

##  Discover and Visualize the Data

# Read the CSV file using pandas first to handle the data
drive.mount('/content/drive')
path = '/content/drive/My Drive/BigData_grp_assignment/apartments10.csv'
df = pd.read_csv(path, sep=None, engine="python", encoding="cp1252")


df.columns = (df.columns
              .str.strip()
              .str.lower()
              .str.replace(r"\s+", "_", regex=True))

# Convert to Spark DataFrame
df_spark = spark.createDataFrame(df)

# Basic data exploration
print("\n=== DATA EXPLORATION ===")
print("Schema:")
df_spark.printSchema()

print("\nBasic statistics:")
df_spark.describe().show()

print("\nData types and null counts:")
df_spark.select([count(when(col(c).isNull(), c)).alias(c) for c in df_spark.columns]).show()

# Show price statistics for understanding distribution
price_stats = df_spark.select("price").describe()
price_stats.show()




Spark Session initialized successfully!
Spark Version: 3.5.1
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

=== DATA EXPLORATION ===
Schema:
root
 |-- id: long (nullable = true)
 |-- category: string (nullable = true)
 |-- title: string (nullable = true)
 |-- body: string (nullable = true)
 |-- amenities: string (nullable = true)
 |-- bathrooms: double (nullable = true)
 |-- bedrooms: double (nullable = true)
 |-- currency: string (nullable = true)
 |-- fee: string (nullable = true)
 |-- has_photo: string (nullable = true)
 |-- pets_allowed: string (nullable = true)
 |-- price: long (nullable = true)
 |-- price_display: string (nullable = true)
 |-- price_type: string (nullable = true)
 |-- square_feet: long (nullable = true)
 |-- address: string (nullable = true)
 |-- cityname: string (nullable = true)
 |-- state: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (n

In [ ]:
## Prepare the Data for Machine Learning Algorithms
print("\n=== Data Preparation ===")

# Start with clean data
print("Basic data cleaning...")
df_clean = df_spark.filter(col('price').isNotNull() & (col('price') > 0))

# Convert price to double to ensure it's numeric
df_clean = df_clean.withColumn("price", col("price").cast("double"))

# Fill missing values with safe defaults
df_clean = df_clean.fillna({
    'bedrooms': 1.0,
    'bathrooms': 1.0,
    'square_feet': 500.0,
    'latitude': 0.0,
    'longitude': 0.0,
    'cityname': 'Unknown',
    'state': 'Unknown',
    'pets_allowed': 'Unknown',
    'has_photo': 'No',
    'source': 'Unknown'
})

# Cast numeric columns to double
numeric_cols = ['bedrooms', 'bathrooms', 'square_feet', 'latitude', 'longitude']
for col_name in numeric_cols:
    if col_name in df_clean.columns:
        df_clean = df_clean.withColumn(col_name, col(col_name).cast("double"))



print("Creating engineered features with RFormula approach")

# Safe feature engineering (avoiding division by zero)
df_clean = df_clean.withColumn("price_per_sqft",
    when(col("square_feet") > 0, col("price") / col("square_feet")).otherwise(0.0))

df_clean = df_clean.withColumn("total_rooms",
    col("bedrooms") + col("bathrooms"))

df_clean = df_clean.withColumn("location_score",
    abs(col("latitude")) + abs(col("longitude")))

# Room density feature
df_clean = df_clean.withColumn("room_density",
    when(col("square_feet") > 0, col("total_rooms") / col("square_feet")).otherwise(0.0))

print("New features created successfully")

print("Creating price categories for classification")

# Create price categories using quantiles
price_quantiles = df_clean.approxQuantile("price", [0.0, 0.33, 0.67, 1.0], 0.05)
print(f"Price boundaries: {price_quantiles}")

# Create bucketizer
bucketizer = Bucketizer(
    splits=price_quantiles,
    inputCol="price",
    outputCol="price_category"
)

df_categorized = bucketizer.transform(df_clean)

# Add readable labels
df_categorized = df_categorized.withColumn("price_label",
    when(col("price_category") == 0.0, "Low")
    .when(col("price_category") == 1.0, "Medium")
    .otherwise("High")
)

print("Price category distribution:")
df_categorized.groupBy("price_category", "price_label").count().orderBy("price_category").show()

print("String indexing for categorical variables")

# Simple string indexing for key categorical columns
categorical_cols = ['state', 'has_photo']
indexers = []
indexed_cols = []

for col_name in categorical_cols:
    if col_name in df_categorized.columns:
        indexer = StringIndexer(
            inputCol=col_name,
            outputCol=f"{col_name}_idx",
            handleInvalid="keep"
        )
        indexers.append(indexer)
        indexed_cols.append(f"{col_name}_idx")

print("String indexers created.")

print("Feature selection...")

# Select reliable features
final_features = [
    'bedrooms', 'bathrooms', 'square_feet',
    'price_per_sqft', 'total_rooms', 'location_score', 'room_density'
] + indexed_cols

print(f"Selected features: {final_features}")

print("6. Train/test split...")

# Split data (80-20)
train_data, test_data = df_categorized.randomSplit([0.8, 0.2], seed=42)

print(f"Training data: {train_data.count()} rows")
print(f"Test data: {test_data.count()} rows")

print("Train set price distribution:")
train_data.groupBy("price_category", "price_label").count().orderBy("price_category").show()


=== BULLETPROOF DATA PREPARATION ===
1. Basic data cleaning...
Missing values handled and types converted.
2. Creating engineered features with RFormula approach...
New features created successfully.
3. Creating price categories for classification...
Price boundaries: [200.0, 1009.0, 1489.0, 52500.0]
Price category distribution:
+--------------+-----------+-----+
|price_category|price_label|count|
+--------------+-----------+-----+
|           0.0|        Low| 3094|
|           1.0|     Medium| 3335|
|           2.0|       High| 3571|
+--------------+-----------+-----+

4. String indexing for categorical variables...
String indexers created.
5. Feature selection...
Selected features: ['bedrooms', 'bathrooms', 'square_feet', 'price_per_sqft', 'total_rooms', 'location_score', 'room_density', 'state_idx', 'has_photo_idx']
6. Train/test split...
Training data: 8053 rows
Test data: 1947 rows
Train set price distribution:
+--------------+-----------+-----+
|price_category|price_label|count|

In [ ]:

print("\n=== NAIVE BAYES MODEL TRAINING ===")

stages = indexers + [
    VectorAssembler(
        inputCols=final_features,
        outputCol="features",
        handleInvalid="skip"  # Skip rows with invalid values
    ),
    NaiveBayes(
        featuresCol="features",
        labelCol="price_category",
        smoothing=1.0
    )
]

pipeline = Pipeline(stages=stages)

print("Training Naive Bayes model...")
try:
    # Train the model
    nb_model = pipeline.fit(train_data)
    print(" Model trained successfully!")

    # Make predictions
    print("Making predictions on test data...")
    nb_predictions = nb_model.transform(test_data)
    print(" Predictions completed!")

except Exception as e:
    print(f" Training failed with error: {e}")




    print("Checking feature columns in training data:")
    for col_name in final_features:
        if col_name in train_data.columns:
            try:
                null_count = train_data.filter(col(col_name).isNull()).count()
                col_type = train_data.select(col_name).dtypes[0][1]
                print(f"   {col_name}: {col_type}, {null_count} nulls")
            except Exception as col_error:
                print(f"   {col_name}: Error - {col_error}")
        else:
            print(f"   {col_name}: MISSING COLUMN")

    print("Sample of training data:")
    train_data.select(final_features[:5] + ["price_category"]).show(5, truncate=False)

    # Try with minimal features if full feature set fails
    print("Trying with minimal feature set...")
    minimal_features = ['bedrooms', 'bathrooms']

    minimal_pipeline = Pipeline(stages=[
        VectorAssembler(
            inputCols=minimal_features,
            outputCol="features",
            handleInvalid="skip"
        ),
        NaiveBayes(
            featuresCol="features",
            labelCol="price_category",
            smoothing=1.0
        )
    ])

    try:
        nb_model = minimal_pipeline.fit(train_data)
        nb_predictions = nb_model.transform(test_data)
        print(" SUCCESS with minimal features!")
        final_features = minimal_features  # Update for evaluation
    except Exception as e2:
        print(f" Even minimal features failed: {e2}")
        raise

## Step 4: Fine-tune the Model (UPDATED SECTION)

print("\n=== MODEL FINE-TUNING ===")

# Create tuning pipeline
nb_tuning = NaiveBayes(featuresCol="features", labelCol="price_category")
tuning_stages = indexers + [
    VectorAssembler(
        inputCols=final_features,
        outputCol="features",
        handleInvalid="skip"
    ),
    nb_tuning
]
tuning_pipeline = Pipeline(stages=tuning_stages)

# Parameter grid for smoothing
paramGrid = ParamGridBuilder() \
    .addGrid(nb_tuning.smoothing, [0.1, 0.5, 1.0, 2.0, 5.0]) \
    .build()

# Cross validator
evaluator = MulticlassClassificationEvaluator(
    labelCol="price_category",
    predictionCol="prediction",
    metricName="accuracy"
)

crossval = CrossValidator(
    estimator=tuning_pipeline,
    estimatorParamMaps=paramGrid,
    evaluator=evaluator,
    numFolds=3
)

print("Fine-tuning model with cross-validation...")
try:
    nb_tuned_model = crossval.fit(train_data)
    nb_tuned_predictions = nb_tuned_model.transform(test_data)

    best_smoothing = nb_tuned_model.bestModel.stages[-1].getSmoothing()
    print(f"Fine-tuning completed! Best smoothing: {best_smoothing}")

except Exception as e:
    print(f"Fine-tuning failed: {e}")
    print("Using original model for evaluation...")
    nb_tuned_predictions = nb_predictions
    best_smoothing = 1.0

## Step 5: Evaluate the Model

print("\n=== MODEL EVALUATION ===")

# Create evaluators
accuracy_evaluator = MulticlassClassificationEvaluator(
    labelCol="price_category", predictionCol="prediction", metricName="accuracy")
precision_evaluator = MulticlassClassificationEvaluator(
    labelCol="price_category", predictionCol="prediction", metricName="weightedPrecision")
recall_evaluator = MulticlassClassificationEvaluator(
    labelCol="price_category", predictionCol="prediction", metricName="weightedRecall")
f1_evaluator = MulticlassClassificationEvaluator(
    labelCol="price_category", predictionCol="prediction", metricName="f1")

# Evaluate models
print("Evaluating model performance...")

# Original model
nb_accuracy = accuracy_evaluator.evaluate(nb_predictions)
nb_precision = precision_evaluator.evaluate(nb_predictions)
nb_recall = recall_evaluator.evaluate(nb_predictions)
nb_f1 = f1_evaluator.evaluate(nb_predictions)

# Tuned model
nb_tuned_accuracy = accuracy_evaluator.evaluate(nb_tuned_predictions)
nb_tuned_precision = precision_evaluator.evaluate(nb_tuned_predictions)
nb_tuned_recall = recall_evaluator.evaluate(nb_tuned_predictions)
nb_tuned_f1 = f1_evaluator.evaluate(nb_tuned_predictions)

# Display results
print("\n=== MODEL PERFORMANCE COMPARISON ===")
print("-" * 80)
print(f"{'Model':<15} {'Accuracy':<12} {'Precision':<12} {'Recall':<12} {'F1-Score':<12}")
print("-" * 80)
print(f"{'Original NB':<15} {nb_accuracy:<12.4f} {nb_precision:<12.4f} {nb_recall:<12.4f} {nb_f1:<12.4f}")
print(f"{'Tuned NB':<15} {nb_tuned_accuracy:<12.4f} {nb_tuned_precision:<12.4f} {nb_tuned_recall:<12.4f} {nb_tuned_f1:<12.4f}")

# Determine best model
if nb_tuned_accuracy > nb_accuracy:
    best_model_name = "Tuned Naive Bayes"
    best_predictions = nb_tuned_predictions
    best_accuracy = nb_tuned_accuracy
else:
    best_model_name = "Original Naive Bayes"
    best_predictions = nb_predictions
    best_accuracy = nb_accuracy

print(f"\n🏆 Best Model: {best_model_name}")
print(f"🎯 Best Accuracy: {best_accuracy:.4f} ({best_accuracy*100:.2f}%)")


=== NAIVE BAYES MODEL TRAINING ===
Building pipeline...
Training Naive Bayes model...
✅ Model trained successfully!
Making predictions on test data...
✅ Predictions completed!

=== MODEL FINE-TUNING ===
Fine-tuning model with cross-validation...
✅ Fine-tuning completed! Best smoothing: 5.0

=== MODEL EVALUATION ===
Evaluating model performance...

=== MODEL PERFORMANCE COMPARISON ===
--------------------------------------------------------------------------------
Model           Accuracy     Precision    Recall       F1-Score    
--------------------------------------------------------------------------------
Original NB     0.4848       0.4837       0.4848       0.4770      
Tuned NB        0.4848       0.4837       0.4848       0.4770      

🏆 Best Model: Original Naive Bayes
🎯 Best Accuracy: 0.4848 (48.48%)
